<a href="https://colab.research.google.com/github/rafiqhasan/auto-tensorflow/blob/main/tutorials/TFAuto_%7C_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install git+https://github.com/rafiqhasan/auto-tensorflow.git
!pip install auto-tensorflow

In [ ]:
from auto_tensorflow.tfa import TFAuto

### **Download data**

In [ ]:
!rm -rf data.*
!rm -rf /content/*.png
!rm -rf *trainer.py
!rm -r /content/train_data
!rm -r /content/test_data
!rm -rf untitled_project
!mkdir /content/train_data
!mkdir /content/test_data
!sudo rm -r /content/tfauto

In [ ]:
# # House price
%%bash
cd /content/train_data
wget https://raw.githubusercontent.com/rafiqhasan/AI_DL_ML_Repo/master/Datasets/house_price/data.csv

cd ../test_data
wget https://raw.githubusercontent.com/rafiqhasan/AI_DL_ML_Repo/master/Datasets/house_price/data.csv ##Taken same data for demonstration purposes only

In [ ]:
##Initialize TFAuto with root and Data path
tfa = TFAuto(train_data_path='/content/train_data/', test_data_path='/content/test_data/', path_root='/content/tfauto')

In [ ]:
##Step 1
##Run Data setup -> Infer Schema, find anomalies, create profile and show viz
tfa.step_data_explore(viz=False)

In [ ]:
##Step 2
##Run Model Training ->
tfa.step_model_build(label_column = 'price', model_type='REGRESSION')     ##--> Default model_complexity
# tfa.step_model_build(label_column = 'price', model_type='REGRESSION', model_complexity=0)     ##--> Model_complexity = 0 ( Simple model - No HPT )

In [ ]:
##Step 3
##Show model What-If Tool
tfa.step_model_whatif()

In [ ]:
#Check signature
!saved_model_cli show --dir "/content/tfauto/model/1" --all

## **Tensorflow Model Serving**

In [ ]:
!apt-get remove tensorflow-model-server
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

!apt-get install tensorflow-model-server

In [ ]:
###Start Tensorflow server
# %%bash --bg 
# export TF_CPP_MIN_VLOG_LEVEL=0

%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8502 \
  --model_name=model \
  --model_base_path="/content/tfauto/model" >server.log 2>&1

In [ ]:
!tail server.log

In [ ]:
import json
import requests

#Create payload
data_py = {"inputs":{'bedrooms': [[3]],
                         'bathrooms': [[2.0]],
                         'sqft_living': [[1180]],
                         'sqft_lot': [[5650]],
                         'floors': [[2.0]],
                         'waterfront': [[1]],
                         'view': [[1]],
                         'condition': [[3]],
                         'grade': [[7]],
                         'sqft_above': [[1180]],
                         'sqft_basement': [[0]],
                         'yr_built': [[1997]],
                         'sqft_living15': [[1340]],
                         'sqft_lot15': [[5650]]
                     }}
                     
data = json.dumps(data_py)
print("payload: ", data)

#Run request on TMS
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8502/v1/models/model:predict', data=data, headers=headers)
json_response.text